In [1]:
import pandas as pd
import numpy as np
import os
import time
from collections import namedtuple
pd.set_option("display.max_rows",35)
%matplotlib inline

In [2]:
a = pd.read_pickle('D:/AI/X_train.pkl')
a1 = pd.read_pickle('D:/AI/X_test.pkl')
b = pd.read_pickle('D:/AI/y_train.pkl')
b1 = pd.read_pickle('D:/AI/y_test.pkl')

In [13]:
from sklearn import model_selection as ms
from sklearn import preprocessing as pp
from sklearn.preprocessing import MinMaxScaler

class preprocess:
    
    output_columns_2labels = ['label_AUDIO','label_BROWSING','label_CHAT','label_FILE-TRANSFER','label_MAIL','label_P2P','label_VIDEO','label_VOIP']
    
    x_input = a.drop(output_columns_2labels, axis = 1)
    y_output = b.loc[:,output_columns_2labels]


    x_test_input = a1.drop(output_columns_2labels, axis = 1)
    y_test = b1.loc[:,output_columns_2labels]
    
    #x_test__input = dataset.kdd_test__2labels.drop(output_columns_2labels, axis = 1)
    #y_test_ = dataset.kdd_test__2labels.loc[:,output_columns_2labels]

    sc = pp.StandardScaler()

    a_train1 = sc.fit_transform(x_input)
    a1_test1 = sc.transform(x_test_input)
    #x_test_ = ss.transform(x_test__input)

    b_train1 = y_output.values
    b1_test1 = y_test.values
    #y_test_ = y_test_.values

    
preprocess.a_train1.std()
print(preprocess.b1_test1)

[[0 0 0 ... 0 0 1]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 1]]


In [4]:
x_train1 = np.reshape(preprocess.a_train1, (preprocess.a_train1.shape[0], preprocess.a_train1.shape[1], 1))

In [5]:
x_train1.shape

(11606, 24, 1)

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

#Initializing the RNN
regressor = Sequential()

#Adding the first LSTM layer and some dropout regularization
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train1.shape[1], 1)))
regressor.add(Dropout(0.2))

#Adding second LSTM layer and some dropout regularization
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

#Adding third LSTM layer and some dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

#Adding fourth LSTM layer and dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

#Adding the output layer
regressor.add(Dense(units = 8))

#compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.summary()


c:\users\anusha\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 24, 50)            10400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 24, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 24, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 24, 50)            20200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 24, 50)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)                20200     
__________

In [7]:
regressor.fit(x_train1, preprocess.b_train1, epochs = 10, batch_size = 1000)

Epoch 1/10
11606/11606 [==============================] - 15s 1ms/step - loss: 0.1116
Epoch 2/10
11606/11606 [==============================] - 12s 1ms/step - loss: 0.0981
Epoch 3/10
11606/11606 [==============================] - 12s 1ms/step - loss: 0.0960
Epoch 4/10
11606/11606 [==============================] - 12s 1ms/step - loss: 0.0931
Epoch 5/10
11606/11606 [==============================] - 12s 1ms/step - loss: 0.0885
Epoch 6/10
11606/11606 [==============================] - 12s 1ms/step - loss: 0.0856
Epoch 7/10
11606/11606 [==============================] - 12s 1ms/step - loss: 0.0828
Epoch 8/10
11606/11606 [==============================] - 12s 1ms/step - loss: 0.0807
Epoch 9/10
11606/11606 [==============================] - 13s 1ms/step - loss: 0.0788
Epoch 10/10
11606/11606 [==============================] - 12s 1ms/step - loss: 0.0758


In [9]:
x_test1 = np.reshape(preprocess.a1_test1, (preprocess.a1_test1.shape[0], preprocess.a1_test1.shape[1], 1))

In [10]:
predictions = regressor.predict(x_test1)

In [15]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = regressor.predict(x_test1)
print(y_pred)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

y_pred = regressor.predict_classes(x_test1)
print(y_pred)

p=regressor.predict_proba(x_test1)

target_names = ['0', '1','2','3','4','5','6','7']
print(classification_report(np.argmax(preprocess.b1_test1, axis=1), y_pred, target_names=target_names, digits=4))
print(confusion_matrix(np.argmax(preprocess.b1_test1, axis=1), y_pred))

[[ 4.77307141e-02  8.95726383e-02  1.87306572e-03 ...  1.30059034e-01
   1.13048933e-01  3.80443692e-01]
 [ 1.32612493e-02  7.24849403e-02  2.97415257e-03 ...  1.47034541e-01
   1.06488720e-01  4.19157803e-01]
 [ 1.91802867e-02  7.78996572e-02 -1.54088438e-03 ...  2.11391468e-02
   3.10763717e-04  8.54429007e-01]
 ...
 [ 1.57205611e-01  4.57769066e-01  1.08645760e-01 ...  3.52093019e-03
   1.01611666e-01  4.11928818e-02]
 [ 1.58312619e-01  4.51331228e-01  1.09562084e-01 ...  1.00418832e-02
   1.10399768e-01  4.18119058e-02]
 [ 2.55795717e-02  6.04174063e-02  1.85254682e-03 ...  1.55286893e-01
   1.05098411e-01  4.43525106e-01]]
[7 7 7 ... 1 1 7]
[7 7 7 ... 1 1 7]
             precision    recall  f1-score   support

          0     0.5000    0.0052    0.0104       191
          1     0.4181    0.8473    0.5599       524
          2     0.0000    0.0000    0.0000       109
          3     0.9079    0.5815    0.7089       356
          4     0.0000    0.0000    0.0000        92
         

c:\users\anusha\appdata\local\programs\python\python36\lib\site-packages\keras\models.py:1116: UserWarning: Network returning invalid probability values. The last layer might not normalize predictions into probabilities (like softmax or sigmoid would).
  warnings.warn('Network returning invalid probability values. '
c:\users\anusha\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
